<a href="https://colab.research.google.com/github/Uzomaaa/TTS_Web_App/blob/main/Web_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
pip install streamlit python-dotenv elevenlabs gradio

In [65]:
import os  # Import the 'os' module to access environment variables
from dotenv import load_dotenv # Import 'load_dotenv' from the 'dotenv' package to read variables from a .env file


# Load environment variables from a .env file into the system environment
load_dotenv()

# Retrieve the value of the 'ELEVEN_API_KEY' variable from the loaded environment
# This will return the API key as a string if it exists, or None if not found
API_KEY = os.getenv("ELEVEN_API_KEY")






In [66]:
# Import required libraries
import gradio as gr         # Gradio is used to build a simple web-based user interface
import requests             # Requests is used to send HTTP requests to the ElevenLabs API


# 🎙️ Define a dictionary of available voices with friendly names and their corresponding voice IDs
VOICE_OPTIONS = {
    "Hank (English Male)": "6F5Zhi321D3Oq7v1oNT4",
    "Andy (English Male)": "D38z5RcWu1voky8WS1ja",
    "Alexandra English Female": "kdmDKE6EkgrWrrykO9Qt"
}

# 🔊 Function to send text to ElevenLabs API and receive an audio response
def generate_audio(text, voice_id, api_key):
    # Construct the API URL using the selected voice ID
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"

    # Define headers required by the API (including the API key for authentication)
    headers = {
        "Accept": "audio/mpeg",              # We want the response as an audio file (MP3)
        "Content-Type": "application/json",  # We're sending JSON data
        "xi-api-key": api_key                # Authentication using our API key
    }

    # Define the payload (the actual data we send) with the text and voice settings
    payload = {
        "text": text,  # The text you want converted to speech
        "model_id": "eleven_multilingual_v2",  # The model to use (supports many languages)
        "voice_settings": {
            "stability": 0.4,          # Controls how consistent the voice sounds
            "similarity_boost": 0.7    # Makes the voice sound closer to its original tone
        }
    }

    # 📡 Send the POST request to the API
    response = requests.post(url, headers=headers, json=payload)

    # 📥 If the request is successful, save the audio response to an MP3 file
    if response.status_code == 200:
        output_path = "output.mp3"               # Local file path for the saved audio
        with open(output_path, "wb") as f:
            f.write(response.content)            # Write binary content to MP3 file
        return output_path                       # Return the path to the saved file
    else:
        # 🚨 If request failed, return the error code and message
        return f"❌ Error: {response.status_code} - {response.text}"

# 🧠 This is the function Gradio will call. It maps a voice name to its ID and generates the audio
def speak(text, selected_voice_name):
    voice_id = VOICE_OPTIONS[selected_voice_name]     # Get the voice ID from the selected name
    return generate_audio(text, voice_id, API_KEY)     # Generate and return the audio file

# 🖼️ Define the Gradio web interface
iface = gr.Interface(
    fn=speak,  # The function to call when user submits input

    # Define the input components: a textbox for the sentence and a dropdown to choose the voice
    inputs=[
        gr.Textbox(label="📝 Enter your text", placeholder="Type or paste your English sentence..."),
        gr.Dropdown(label="🗣️ Choose a Voice", choices=list(VOICE_OPTIONS.keys()), value="Hank (English Male)")
    ],

    # Define the output: an audio player to play and download the result
    outputs=gr.Audio(label="▶️ Generated Speech", type="filepath"),

    # Optional UI settings
    title="🗣️ English Text to Speech App",
    description="""
    <h3>🔊 Convert English Text to Natural Speech</h3>
    <p>Type your sentence, choose a voice, and hear it spoken aloud!
    """,
    theme="default"
)

# 🚀 Launch the app and generate a shareable public URL
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29c15fee56eea08cba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
